In [1]:
import networkx as nx
import random
import json
import os
import ast  # 用于安全地 eval 节点字符串

import os
import random
import requests
import time
from PIL import Image
import base64
import io
import json
import re
from tqdm import tqdm
import numpy as np

In [2]:
import csv
import random
import time
import json
import copy


import os
os.environ['HTTP_PROXY'] = 'http://127.0.0.1:7890'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'
os.environ['LANGSMITH_TRACING'] = 'true'
os.environ['LANGSMITH_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGSMITH_API_KEY'] = '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'
os.environ['LANGSMITH_PROJECT'] = 'pr-wooden-hedgehog-65'
os.environ['OPENAI_API_KEY'] = 'sk-g4MycviIFf8Lad0v5zVKPljDpueIDBLuqIC1nsDAiRnLZVKg'
os.environ['OPENAI_API_BASE'] = 'https://api2.aigcbest.top/v1'

from typing import Any, List, Dict

# LangChain 和 LangSmith 的相关导入
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, ChatMessage

def generate_from_openai_chat_completion_new(
    messages: List[Dict[str, str]],
    model: str,
    temperature: float,
    max_tokens: int,
    top_p: float,
    context_length: int, # 这个参数在ChatOpenAI中不直接使用，但为了保持签名一致而保留
    stop_token: str | None = None,
) -> str:
    """
    使用 LangChain 的 ChatOpenAI 生成聊天回复，并支持 LangSmith 追踪。
    函数签名与旧版完全兼容。
    """
    if "OPENAI_API_KEY" not in os.environ and "api_key" not in locals():
        raise ValueError("OPENAI_API_KEY environment variable must be set.")

    # 1. 初始化 LangChain 的 ChatOpenAI 客户端
    # 将旧函数的参数映射到 ChatOpenAI 的构造函数中
    llm = ChatOpenAI(
        # 从环境变量获取 key 和 base_url，提供默认值
        api_key=os.environ.get("OPENAI_API_KEY"),
        base_url=os.environ.get("OPENAI_API_BASE", "https://api.openai.com/v1"),
        model=model,
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p,
        stop=[stop_token] if stop_token else None,
        model_kwargs={}, # 如果没有其他高级参数，可以为空字典或直接省略
        # 内置重试机制，替代旧的装饰器
        max_retries=3,
    )

    # 2. 将输入的字典列表转换为 LangChain 的消息对象
    # 这是一个必要的适配步骤
    langchain_messages = []
    for msg in messages:
        role = msg.get("role")
        content = msg.get("content", "")
        # 从字典中获取 name，如果不存在则为 None
        name = msg.get("name")

        if role == "user":
            langchain_messages.append(HumanMessage(content=content, name=name))
        elif role == "assistant":
            langchain_messages.append(AIMessage(content=content, name=name))
        elif role == "system":
            # 这会正确处理您提供的 few-shot 示例
            langchain_messages.append(SystemMessage(content=content, name=name))
        else:
            # 为其他可能的角色（如 'tool'）提供一个通用处理
            langchain_messages.append(ChatMessage(role=role, content=content))

    # 3. 调用 LLM 并获取结果
    response = llm.invoke(langchain_messages)

    # 4. 提取并返回内容，保持与旧函数相同的字符串输出
    return response.content

In [3]:
# example_messages_with_name = [{
#     'role': 'user',
#     'content': "hello"
# }]

# result = generate_from_openai_chat_completion_new(
#     messages=example_messages_with_name,
#     model="gpt-5.1",
#     temperature=0.7,
#     max_tokens=2000,
#     top_p=1.0,
#     context_length=4096,
#     stop_token=None
# )
# result

In [4]:
system_prompt = """You are a User Behavior Simulator acting as a Router.
Your goal is to navigate a website and synthesize a coherent high-level task.

### INPUTS
1. **Interaction History**: Steps taken so far.
2. **Current High-Level Intent**: The hypothesized goal of the user based on previous actions. If not provided, you should generate a new one based on the history.
3. **Current Page Accessibility Tree**: The structure of the current page.
4. **Available Options**: A list of candidate next actions (edges in the graph).
5. **Remaining Steps**: The number of steps left to complete the task.

### INSTRUCTIONS
1. **Analyze**: Review the history and the current intent.
2. **Maintain Consistency**: You should prioritizing following the 'Current High-Level Intent' if it remains feasible and logical. Only shift the intent if the current page offers no valid actions to support it, or if the intent was clearly exploratory/random.
3. **Select**: Choose the most logical option from the 'Available Options' list that advances the user towards the intent.
4. **Refine**: If needed, refine the 'High-Level Intent' to be more specific based on the chosen action.
5. **Step Count**: You should also consider the 'Remaining Steps' to complete the task.

### EXAMPLE ABSTRACT INTENTION
"Upvote or downvote submissions by a specific user or based on ranking criteria (e.g., newest, top-rated).",
"Ask for personal advice or relationship guidance in an appropriate community.",
"Request product recommendations within a specific budget and category.",
"Initiate a discussion about a specific technical topic or emerging technology field.",
"Post a review of a book or media content with personal commentary.",
"Announce a virtual meetup or event for enthusiasts of a specific interest.",
"Crosspost an image or content to a different community with a source note.",
"Reply to a specific post or comment thread with detailed text.",
"Update user profile details such as the bio description.",
"Edit the content of an existing post to append new information.",
"Retrieve statistics or analyze specific details about comments and user activities in a forum.",
"Navigate to a trending thread and subscribe to the post.",
"Create a new community forum with a specific name, description, and sidebar settings."

### OUTPUT FORMAT
Return a valid JSON object:
{
  "reasoning": "Brief analysis of why this option fits the intent.",
  "refined_high_level_intent": "The updated high-level goal string.",
  "selected_option_index": <Integer index of the chosen option from the list, or -1 if none are suitable>
}
"""

In [5]:
with open("/home/zjusst/qms/webarena/result_stage_3_generation/flitered_triplets_threshold_5.csv", "r") as f:
    reader = csv.reader(f)
    # 跳过第一行
    next(reader)
    triplets = [row for row in reader]
print(len(triplets))
print(triplets[0])

1998
["('/home/zjusst/qms/webarena/result_stage_1_explore_v2/add_local_intention_trajs/812_36.json', 0, 0, 'http://reddit.com/')", "click [20] where [20] is button 'Sort by: Hot' hasPopup: menu expanded: False", 'The user wants to sort posts by a different criterion to better organize or view the content on the homepage.', "('/home/zjusst/qms/webarena/result_stage_1_explore_v2/add_local_intention_trajs/812_36.json', 0, 1, 'http://reddit.com/')"]


In [6]:
def get_origin_traj_json(filepath, index):
    with open(os.path.join(filepath), "r") as f:
        origin_traj_json = json.load(f)
    return origin_traj_json[index]

#get_origin_traj_json("812_1.json", 0)

In [7]:
# outgoing_edges
# # 如果 len(outgoing_edges) > 20，随机选择20个
# if len(outgoing_edges) > 20:
#     outgoing_edges = random.sample(outgoing_edges, 20)


# options_str = ""

# for i in range(len(outgoing_edges)):
#     u, v, key = outgoing_edges[i]
#     edge_info = G.get_edge_data(u, v, key=key)
#     options_str += f"""Option {i}:
# Action: {edge_info['action']}
# Local Intent: {edge_info['intent']}

# """

# print(options_str)

# history_str = ""
# for step in trajectory:
#     real_url = eval(step[0])[3]
#     action = step[1]
#     intention = step[2]
#     history_str += f"Page: {real_url}\tAction: {action}\tIntention: {intention}\n"
    
# print(history_str)

In [8]:
def build_user_prompt(trajectory, current_high_level_inten, outgoing_edges, remaining_steps):
    history_str = ""
    for step in trajectory:
        real_url = eval(step[0])[3]
        action = step[1]
        intention = step[2]
        history_str += f"Page: {real_url}\tAction: {action}\tIntention: {intention}\n"


    current_intent = current_high_level_intent

    if len(outgoing_edges) > 20:
        outgoing_edges = random.sample(outgoing_edges, 20)
    options_str = ""

    for i in range(len(outgoing_edges)):
        u, v, key = outgoing_edges[i]
        edge_info = G.get_edge_data(u, v, key=key)
        options_str += f"""Option {i}:
Action: {edge_info['action']}
Local Intent: {edge_info['intent']}

"""

    filepath, index, _, _ = eval(trajectory[-1][0])
    orgin_json = get_origin_traj_json(filepath, index)
    current_a11y = orgin_json["a11y_after"]

    user_prompt = f"""
### INTERACTION HISTORY
{history_str}

### CURRENT HIGH-LEVEL INTENT
{current_intent}

### CURRENT PAGE ACCESSIBILITY TREE
{current_a11y[:4000]}

### AVAILABLE OPTIONS
{options_str}
### REMAINING STEPS
{remaining_steps}

### YOUR DECISION
Based on the intent and page state, select the best option index (0-{len(outgoing_edges)-1} or -1 if none are suitable)."""

    return user_prompt


graph_data = triplets
G = nx.MultiDiGraph()
for source, action, intent, target in graph_data:
    G.add_edge(source, target, action=action, intent=intent)

current_node = random.choice(list(G.nodes()))

trajectory = []
current_high_level_intent = "None (Just started)" # 初始意图
print(f"Agentic Walk Start. Node: {current_node}")

path_length = 3
warmup_steps = 2

for i in range(path_length):
    
    # 获取所有出边 (source, target, key)
    outgoing_edges = list(G.out_edges(current_node, keys=True))
    print(len(outgoing_edges))
    
    if not outgoing_edges:
        print("Dead end reached.")
        break

    selected_edge_data = None
    next_node_candidate = None

    # --- Phase A: 随机预热 (Warm-up) ---
    if i < warmup_steps:
        # 随机选择
        u, v, key = random.choice(outgoing_edges)
        selected_edge_data = G.get_edge_data(u, v, key=key)
        next_node_candidate = v
        
    else:
        user_prompt = build_user_prompt(trajectory, current_high_level_intent, outgoing_edges, path_length - i)
        print(user_prompt)
        break


    step_tuple = (current_node, selected_edge_data['action'], selected_edge_data['intent'], next_node_candidate)
    trajectory.append(step_tuple)
    
    # 移动到下一个节点
    current_node = next_node_candidate

Agentic Walk Start. Node: ('/home/zjusst/qms/webarena/result_stage_2_intended_explore/add_local_intention_trajs/822.json', 5, 1, 'http://reddit.com/create_forum')
1
70
70

### INTERACTION HISTORY
Page: http://reddit.com/create_forum	Action: click [2917] where [2917] is link 'Submit'	Intention: The user aims to transition from forum creation to the submission creation page, likely intending to submit content to an existing forum.
Page: http://reddit.com/submit	Action: type [3384] [Grappling Techniques: Exploration and Mastery ] where [3384] is textbox 'Title *' required: True	Intention: The user intends to create a new submission by entering a title, specifically about 'Grappling Techniques: Exploration and Mastery'.


### CURRENT HIGH-LEVEL INTENT
None (Just started)

### CURRENT PAGE ACCESSIBILITY TREE
Tab 0 (current): Create submission

[1818] RootWebArea 'Create submission' focused: True
	[1852] HeaderAsNonLandmark ''
		[1853] link 'Home'
			[1880] StaticText 'Postmill'
	[1881] link

In [9]:
def read_json_result(result):
    """
    解析类似```json
    {
    "reason": "The task trajectory includes a sequence of actions with a clear and logical progression towards the goal of accessing the edit history of a wiki page, making edits, and publishing them. Each action is well-documented with a specific local intent, contributing to a coherent sequence. The actions of navigating to the page history, entering the edit page, scrolling to view options, and saving changes are necessary steps to complete the task. The trajectory successfully implements the task by showing the final page with updated content, indicating proper task completion. As for task difficulty, the trajectory spans across a moderate number of steps and involves page navigation, editing, and saving changes, reflecting a medium level of complexity.",
    "score_coherence": 5,
    "score_completion": 5,
    "score_difficulty": 3
    }
    ```
    """

    if not result.startswith("```json"):
        # 直接解析
        try:
            return json.loads(result)
        except json.JSONDecodeError:
            raise ValueError(f"Invalid JSON format in result: {result}")
    else:
        # 从```json```中解析
        json_str = result.split("```json")[1]
        json_str = json_str[:-3]
        try:
            return json.loads(json_str)
        except json.JSONDecodeError:
            raise ValueError(f"Invalid JSON format in result: {result}")

In [10]:
# example_messages_with_name = [
#     {
#         'role': 'system',
#         'content': system_prompt
#     },
#     {
#         'role': 'user',
#         'name': 'user',
#         'content': user_prompt
#     }
# ]

# result = generate_from_openai_chat_completion_new(
#     messages=example_messages_with_name,
#     model="gpt-5.1",
#     temperature=0.7,
#     max_tokens=2000,
#     top_p=1.0,
#     context_length=8192,
#     stop_token=None
# )

# result_json = read_json_result(result)

In [11]:
def filter_outgoing_edges(outgoing_edges):
    flitered_outgoing_edges = []
    action_set = set()

    for outgoing_edge in outgoing_edges:
        u, v, key = outgoing_edge
        edge_info = G.get_edge_data(u, v, key=key)
        action = edge_info['action']
        # 去掉 action 中 [数字] 的部分
        action = re.sub(r'\[.*?\]', '', action)
        if action not in action_set:
            action_set.add(action)
            flitered_outgoing_edges.append(outgoing_edge)

    return flitered_outgoing_edges

def fliter_same_action(outgoing_edges, trajectory):
    flitered_outgoing_edges = []
    action_set = set()
    for step in trajectory:
        action = step[1]
        action_set.add(action)
        
    for outgoing_edge in outgoing_edges:
        u, v, key = outgoing_edge
        edge_info = G.get_edge_data(u, v, key=key)
        action = edge_info['action']
        if action not in action_set:
            flitered_outgoing_edges.append(outgoing_edge)
    return flitered_outgoing_edges

In [12]:
def agentic_walk(triplets, path_length=3, warmup_steps=1):
    graph_data = triplets
    G = nx.MultiDiGraph()
    for source, action, intent, target in graph_data:
        G.add_edge(source, target, action=action, intent=intent)

    current_node = random.choice(list(G.nodes()))

    trajectory = []
    current_high_level_intent = "None (Just started)" # 初始意图
    print(f"Agentic Walk Start. Node: {current_node}")

    for i in range(path_length):
        
        # 获取所有出边 (source, target, key)
        outgoing_edges = list(G.out_edges(current_node, keys=True))
        outgoing_edges = filter_outgoing_edges(outgoing_edges)  # 过滤重复的 action
        outgoing_edges = fliter_same_action(outgoing_edges, trajectory)
        
        if not outgoing_edges:
            print("Dead end reached.")
            break

        selected_edge_data = None
        next_node_candidate = None

        # --- 随机预热 (Warm-up) ---
        if i < warmup_steps:
            # 随机选择
            u, v, key = random.choice(outgoing_edges)
            selected_edge_data = G.get_edge_data(u, v, key=key)
            next_node_candidate = v
            
        else:
            user_prompt = build_user_prompt(trajectory, current_high_level_intent, outgoing_edges, path_length - i)
            example_messages_with_name = [
                {
                    'role': 'system',
                    'content': system_prompt
                },
                {
                    'role': 'user',
                    'name': 'user',
                    'content': user_prompt
                }
            ]

            result = generate_from_openai_chat_completion_new(
                messages=example_messages_with_name,
                model="gpt-5.1",
                temperature=0.7,
                max_tokens=2000,
                top_p=1.0,
                context_length=8192,
                stop_token=None
            )

            result_json = read_json_result(result)
            idx = result_json["selected_option_index"]

            if idx != -1 and 0 <= idx < len(outgoing_edges):
                u, v, key = outgoing_edges[idx]
                selected_edge_data = G.get_edge_data(u, v, key=key)
                next_node_candidate = v
                
                # 更新意图
                current_high_level_intent = result_json.get("refined_high_level_intent", current_high_level_intent)
                print(f"  [Step {i}] Router chose option {idx}. Intent updated to: {current_high_level_intent}")
            else:
                # LLM 认为没有合适的边，或者是输出格式错误
                # 策略：回溯或者直接停止。这里为了简单，选择停止。
                print(f"  [Step {i}] Router found no suitable path. Stopping.")
                break


        step_tuple = (current_node, selected_edge_data['action'], selected_edge_data['intent'], next_node_candidate)
        trajectory.append(step_tuple)
        
        # 移动到下一个节点
        current_node = next_node_candidate

    return trajectory

In [13]:
trajectory

for i in range(17):
    path_length = 4
    warmup_steps = 1

    pass_flag = False

    while not pass_flag:
        trajectory = agentic_walk(triplets, path_length=path_length, warmup_steps=warmup_steps)
        if len(trajectory) == path_length:
            pass_flag = True
        else:
            print(f"  [Step {i}] Router found no suitable path. Retrying.")

    # index 转为占3位的str
    index_str = f"{i+1:03d}"
    with open(f"/home/zjusst/qms/webarena/result_stage_3_generation/selected_router_trajs_v2/{index_str}_len{path_length}.json", "w") as f:
        json.dump({
            "selected_trajectory": trajectory,
        }, f)


Agentic Walk Start. Node: ('/home/zjusst/qms/webarena/result_stage_1_explore_v2/add_local_intention_trajs/812_36.json', 11, 1, 'http://reddit.com/search?q=new+features')
  [Step 1] Router found no suitable path. Stopping.
  [Step 0] Router found no suitable path. Retrying.
Agentic Walk Start. Node: ('/home/zjusst/qms/webarena/result_stage_2_intended_explore/add_local_intention_trajs/873.json', 0, 0, 'http://reddit.com/')
  [Step 1] Router chose option 1. Intent updated to: Find and read discussions or posts about the impact of tech startups on society, possibly by browsing relevant forums if direct search yields no results.
  [Step 2] Router chose option 3. Intent updated to: Browse and possibly participate in a general discussion forum where they can read or discuss topics such as the societal impact of tech startups.
  [Step 3] Router chose option 0. Intent updated to: Create a new AskReddit post related to societal topics such as the impact of tech startups.
Agentic Walk Start. Node